In [1]:
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from get_aq_data import get_flo_data, ID_to_name, TICKS_TWO_HOURLY

In [3]:
data, hourly_mean, daily_mean = get_flo_data()

In [15]:
subset = data['2019-05-01':]
subset = subset.mean(axis=1)
subset['2019-07-01':] = subset['2019-07-01':] + 5
just_sundays = subset[subset.index.dayofweek == 6].to_frame('Mean')

In [16]:
just_sundays['Special'] = 'Non-Special'
just_sundays.loc['2019-05-05', 'Special'] = 'Marathon'
just_sundays.loc['2019-07-21', 'Special'] = 'Just Ride'

In [17]:
just_sundays['Time'] = just_sundays.index.time
just_sundays['Date'] = just_sundays.index.date

In [18]:
pivoted = just_sundays.pivot(columns='Date', index='Time', values='Mean')

In [19]:
pivoted.index[0]

datetime.time(0, 0)

In [20]:
mean_middle_of_day = pivoted[datetime.time(10, 0):datetime.time(14,0)].mean(axis=0)
mean_middle_of_day.name = 'Mean'

In [21]:
day_ids = just_sundays.pivot(columns='Date', index='Time', values='Special').mode(axis=0).loc[0]

In [22]:
day_ids.name = 'Special'

In [23]:
merged = pd.merge(day_ids, mean_middle_of_day, left_index=True, right_index=True)

In [24]:
merged = merged.sort_values('Mean', ascending=False)

In [25]:
merged

,Special,Mean
Date,,
2019-06-23,Non-Special,14.518118
2019-08-04,Non-Special,14.130235
2019-06-02,Non-Special,12.200425
2019-07-14,Non-Special,11.677239
2019-08-11,Non-Special,8.546065
2019-05-19,Non-Special,7.963257
2019-07-07,Non-Special,7.920588
2019-07-28,Non-Special,7.819203
2019-07-21,Just Ride,6.779003


In [51]:
merged.to_excel('SundayRankings.xlsx')